In [1]:
from konlpy.tag import Twitter
import nltk

twitter = Twitter()

print(twitter.morphs(u'한글형태소분석기 테스트 중 입니다')) # ??
print(twitter.nouns(u'한글형태소분석기 테스트 중 입니다!')) #명사
print(twitter.pos(u'한글형태소분석기 테스트 중 입니다.')) #형태소

C:\Users\kim\.conda\envs\exam_cv2\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['한글', '형태소', '분', '석기', '테스트', '중', '입니다']
['한글', '형태소', '석기', '테스트', '중']
[('한글', 'Noun'), ('형태소', 'Noun'), ('분', 'Suffix'), ('석기', 'Noun'), ('테스트', 'Noun'), ('중', 'Noun'), ('입니다', 'Adjective'), ('.', 'Punctuation')]


In [7]:
def read_data(filename):
    with open(filename, 'r', encoding = "utf-8") as f:
        data = [line.split('\t') for line in f.read().splitlines()]
    return data

def tokenize(doc):
  # norm, stem은 optional
  return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

In [8]:
# 트래이닝 데이터와 테스트 데이터를 읽기
train_data = read_data('./data/ratings_train.txt')
test_data = read_data('./data/ratings_test.txt')

# row, column의 수가 제대로 읽혔는지 확인
print(len(train_data))      # nrows: 150000
print(len(train_data[0]))   # ncols: 3
print(len(test_data))       # nrows: 50000
print(len(test_data[0]))     # ncols: 3


150001
3
50001
3


In [9]:
# 형태소 분류
train_docs = [(tokenize(row[1]), row[2]) for row in train_data[1:]]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data[1:]]

#Training data의 token 모으기
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

# Load tokens with nltk.Text()
text = nltk.Text(tokens, name='NMSC')
print(text.vocab().most_common(10))

2159921
[('./Punctuation', 67778), ('영화/Noun', 50818), ('하다/Verb', 41209), ('이/Josa', 38540), ('보다/Verb', 38538), ('의/Josa', 30188), ('../Punctuation', 29055), ('가/Josa', 26627), ('에/Josa', 26468), ('을/Josa', 23118)]


In [10]:
# 텍스트간의 연어 빈번하게 등장하는 단어 구하기
text.collocations()

이/Determiner 것/Noun; 적/Suffix 인/Josa; 이/Determiner 거/Noun; 것/Noun
은/Josa; 10/Number 점/Noun; 배우/Noun 들/Suffix; 이/Noun 게/Josa; 수/Noun
있다/Adjective; 내/Noun 가/Josa; 최고/Noun 의/Josa; 네/Suffix 요/Josa; 이/Noun
영화/Noun; 들/Suffix 이/Josa; 끝/Noun 까지/Josa; 때문/Noun 에/Josa; 적/Suffix
으로/Josa; 못/VerbPrefix 하다/Verb; 사람/Noun 들/Suffix; 1/Number 점/Noun;
영화/Noun 를/Josa


In [11]:
# term이 존재하는지에 따라서 문서를 분류
selected_words = [f[0] for f in text.vocab().most_common(2000)] # 여기서는 최빈도 단어 2000개를 피쳐로 사용
train_docs = train_docs[:10000] # 시간 단축을 위한 꼼수로 training corpus의 일부만 사용할 수 있음
train_xy = [(term_exists(d), c) for d, c in train_docs]
test_xy = [(term_exists(d), c) for d, c in test_docs]

# nltk의 NaiveBayesClassifier으로 데이터를 트래이닝 시키고, test 데이터로 확인
classifier = nltk.NaiveBayesClassifier.train(train_xy) #Naive Bayes classifier 적용
print(nltk.classify.accuracy(classifier, test_xy))
# => 0.80418

classifier.show_most_informative_features(10)


0.80414
Most Informative Features
         exists(수작/Noun) = True                1 : 0      =     38.0 : 1.0
     exists(이딴/Modifier) = True                0 : 1      =     32.1 : 1.0
         exists(최악/Noun) = True                0 : 1      =     30.1 : 1.0
       exists(♥/Foreign) = True                1 : 0      =     24.5 : 1.0
         exists(노잼/Noun) = True                0 : 1      =     22.1 : 1.0
         exists(짜증/Noun) = True                0 : 1      =     19.5 : 1.0
        exists(쓰레기/Noun) = True                0 : 1      =     19.4 : 1.0
         exists(여운/Noun) = True                1 : 0      =     18.9 : 1.0
          exists(굿/Noun) = True                1 : 0      =     17.1 : 1.0
        exists(발연기/Noun) = True                0 : 1      =     16.9 : 1.0
